# Setup

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2


In [2]:
from simply_tiles.vector_tiler import VectorTiler

# TESTS

In [3]:
tiler = VectorTiler(config="my_data/tilecache_config_userbounds.json")

tiler.user_bounds
tiler.bbox

BBOX is set to user defined bounds and reprojected if required.


{'xmin': 1113000,
 'ymin': 7080000,
 'xmax': 1183000,
 'ymax': 7100000,
 'srid': 3857}

{'xmin': 1113000.0,
 'ymin': 7080000.0,
 'xmax': 1183000.0,
 'ymax': 7100000.0,
 'srid': '3857'}

# Create example vector tile cache in WebMercatorQuad

In [16]:
# input parameters
tileset_name = "tilecache_tms_integrated"
tileset_path = "C:/Users/Notna/Desktop/test_caches"
config_path = "my_data/tilecache_config.json"

# instantiate tiler object
tiler = VectorTiler(config_path)

BBOX is auto-detected across all geoms of all specified layers.


In [17]:
# generate tiles
tiler.generate_tileset(tileset_path, tileset_name, max_zoom=15)

  0%|          | 0/2 [00:00<?, ?it/s]

Created temp union table from all specified tables
Geoms deviating from specified pbf_srid are automatically transformed
Generating tiles for zoom level 0
Generating tiles for zoom level 1
Generating tiles for zoom level 2
Generating tiles for zoom level 3
Generating tiles for zoom level 4
Generating tiles for zoom level 5
Generating tiles for zoom level 6
Generating tiles for zoom level 7


  0%|          | 0/10 [00:00<?, ?it/s]

Generating tiles for zoom level 8
Generating tiles for zoom level 9


 31%|███▏      | 10/32 [00:00<00:00, 94.34it/s]

...

  0%|          | 13/4545 [00:00<00:37, 121.49it/s]

Generating tiles for zoom level 14


  0%|          | 13/17889 [00:00<02:18, 128.70it/s]

Generating tiles for zoom level 15


100%|██████████| 17889/17889 [02:00<00:00, 148.17it/s]
